In [8]:
import json

In [4]:
import requests
res = requests.get('http://localhost:9200')

In [5]:
# connect to es cluster
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [14]:
setting_dbl = {
    "settings": {
    "index": {
      "analysis": {
        "analyzer": {
          "my_analyzer": {
            "tokenizer": "standard",
            "filter": [
              "standard",
              "lowercase",
              "dbl_metaphone"
            ]
          }
        },
        "filter": {
          "dbl_metaphone": {
            "type": "phonetic",
            "encoder": "double_metaphone",
            "replace": False}
        }
      }
    }
  }
}

In [19]:
es.indices.create(index='dbl', ignore=400, body=setting_dbl)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'dbl_test'}

In [9]:
r = requests.get('http://localhost:9200')
i = 18
while r.status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+ str(i))
    es.index(index='sw', doc_type='people', id=i, body=json.loads(r.content))
    i=i+1

In [10]:
es.search(index='sw', body={'query': {'match': {'name':'Darth Vader'}}})

{'took': 62,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 1,
  'max_score': 2.2269652,
  'hits': [{'_index': 'sw',
    '_type': 'people',
    '_id': '44',
    '_score': 2.2269652,
    '_source': {'name': 'Darth Maul',
     'height': '175',
     'mass': '80',
     'hair_color': 'none',
     'skin_color': 'red',
     'eye_color': 'yellow',
     'birth_year': '54BBY',
     'gender': 'male',
     'homeworld': 'https://swapi.co/api/planets/36/',
     'films': ['https://swapi.co/api/films/4/'],
     'species': ['https://swapi.co/api/species/22/'],
     'vehicles': ['https://swapi.co/api/vehicles/42/'],
     'starships': ['https://swapi.co/api/starships/41/'],
     'created': '2014-12-19T18:00:41.929000Z',
     'edited': '2014-12-20T21:17:50.403000Z',
     'url': 'https://swapi.co/api/people/44/'}}]}}